**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Optimal taxation with government consumption](#toc1_)    
- 2. [Problem 2: Labor adjustment costs](#toc2_)    
- 3. [Problem 3: Global optimizer with refined multi-start](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
from itertools import product
import numpy as np
from scipy import optimize
from scipy import interpolate

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.style.use('seaborn')
import seaborn as sns
from IPython.display import display

## 1. <a id='toc1_'></a>[Problem 1: Optimal taxation with government consumption](#toc0_)


Consider a worker choosing hours of labor, $L\in[0,24]$, to maximize utility: 

$$
\begin{align*}
V(w,\tau,G)&=\max_{L\in[0,24]}\ln\left(C^{\alpha}G^{1-\alpha}\right)-\nu\frac{L^{2}}{2}\\&\text{s.t.}\\&C=\kappa+(1-\tau)wL
\end{align*}
$$

where 

* $C$ is *private* consumption with weight $\alpha\in(0,1)$.
* $\kappa > 0$ is the *free private* consumption component.
* $(1-\tau)wL$ is the *costly private* consumption component.
* $w > 0 $ is the real wage.
* $\tau \in (0,1)$ is the labor-income tax rate.
* $G > 0 $ is *government* consumption with weight $1-\alpha$.
* $\nu > 0$ is the disutility of labor scaling factor


The baseline parameters are:

$$
\begin{align*}
\alpha &= 0.5\\
\kappa &= 1.0\\
\nu &= \frac{1}{2\cdot16^2} \\
w &= 1.0 \\ 
\tau &= 0.30 \\
\end{align*}
$$

**Question 1:** Verify that the optimal labor supply choice is $L^{\star}(\tilde{w}) =\frac{-\kappa+\sqrt{\kappa^{2}+4\frac{\alpha}{\nu}\tilde{w}^2}}{2\tilde{w}}$, where $\tilde{w} = (1-\tau)w$, for $G\in\left\{1.0 , 2.0\right\}$.

In [ ]:
# Create an instance of the LaborSupplyModel class
from Exam import LaborSupplyModel

# Create an instance of the LaborSupplyModel class
model = LaborSupplyModel()

# Display the equation for the optimal labor supply choice
display(model.display_optimal_labor_supply_equation())




Above is verifyed that the optimal labor supply is given by L^*(w^tilde)

**Question 2:** Illustrate how $L^{\star}(\tilde{w})$ depends on $w$.

In [ ]:
from Exam import LaborSupplyGraph
import numpy as np

alpha = 0.5
kappa = 1.0
nu = 1.0 / (2 * 16 ** 2)
w = 1.0
tau = 0.30
G_values = [1.0]

# Create an instance of the LaborSupplyModel
model = LaborSupplyGraph(alpha, kappa, nu, tau)

# Generate range of wage values
w_values = np.linspace(0.1, 3.0, 100)  # Adjust the range according to your needs

# Plotting
model.plot_labor_supply_graph(w_values, G_values)

The above figure shows the optimal labor supply against the wage. As the wage increases the optimal labor supply will increase. When the wage is between 0.5 and 1 the curve of the optimal labor supply will flatten, and as wage increase the optimal labor supply will stay around the same level. 


We now consider a government, who chooses $\tau$ and spend all of the taxes on government consumption so:

$$
G = \tau w L^{\star}((1-\tau)w)
$$

**Question 3:** Plot the implied $L$, $G$ and worker utility for a grid of $\tau$-values.


In [ ]:
# write your code here
from Exam import LaborSupplyGraphQ3
alpha = 0.5
kappa = 1.0
nu = 1.0 / (2 * 16 ** 2)
w = 1.0
tau_values = np.linspace(0, 1, 100)  # Grid of tau values


# Create an instance of the modified LaborSupplyGraph class
model = LaborSupplyGraphQ3(alpha, kappa, nu)

# Plot the implied values for the grid of tau values
model.plot_implied_values(w, tau_values)

The graph to the left shows the relationship between labor supply and the tax rate. There is a negative relationship between the two parameters since a higher tax rate makes it less profitable to work. 

The graph in the middle shows the relationship beetween government spending and the tax rate. A higher tax rate has a positive effect on the government consumption when it is not too high. When the tax rate becomes too high there are no incentive to work, which lowers the government consumption. 

The graph to the right shows the relationship between worker utility and the tax rate. The worker gets advantage of a low tax however the incentive to work decreases when the tax rate becomes high. 

**Question 4:** Find the socially optimal tax rate $\tau^{\star}\in(0,1)$ maximizing worker utility. Illustrate your result.

In [ ]:
# write your code here
from Exam import LaborSupplyGraphQ4
import numpy as np

alpha = 0.5
kappa = 1.0
nu = 1.0 / (2 * 16 ** 2)
w = 1.0
tau_range = np.linspace(0.0, 1.0, 100)  # Grid of tau values

# Create an instance of the modified LaborSupplyGraphQ4 class
model = LaborSupplyGraphQ4(alpha, kappa, nu)

# Plot the worker utility and find the socially optimal tax rate
model.plot_optimal_tax_rate(w, tau_range)

The graph above shows that the socially optimal tax rate is 0.515. The worker utility will increase until the point where the tax rate is 0.515, after this point the worker utility will decrease.

A more general preference formulation for the worker is:

$$
\begin{align*}
\mathcal{V}(w,\tau,G)&=\max_{L\in[0,24]}\frac{\left[ \left( \alpha C^{\frac{\sigma-1}{\sigma}}+(1-\alpha) G^{\frac{\sigma-1}{\sigma}} \right)^{\frac{\sigma}{1-\sigma} }\right]^{1-\rho}-1}{1-\rho}- \nu\frac{L^{1+\varepsilon}}{1+\varepsilon},\,\,\,\varepsilon,\rho,\sigma>0,\,\,\,\rho,\sigma\neq1\\&\text{s.t.}\\&C=\kappa+(1-\tau)wL
\end{align*}    
$$

Optimal labor supply is now $L^{\star}(\tilde{w},G)$.

Questions 5 and 6 must be answered with the general formulation, and for 2 different set of parameters:

- Set 1:  $\sigma = 1.001$, $\rho = 1.001$ and $\varepsilon = 1.0$.
- Set 2:  $\sigma = 1.5$, $\rho = 1.5$ and $\varepsilon = 1.0 $.

**Question 5:** Find the $G$ that solves $G = \tau w L^{\star}((1-\tau)w,G)$ using the $\tau$ found in question 4.

*Hint: First write code that solves the worker problem for given values of $G$ and $\tau$. Then find the correct G based on this.*

In [ ]:
from Exam import LaborSupplyGraphQ5

alpha = 0.5
kappa = 1.0
nu = 1.0/(1*16**2)
sigma = 1.001
rho = 1.001
epsilon = 1.0

w = 10.0
tau = 0.5151515151515152

model = LaborSupplyGraphQ5(alpha, kappa, nu, sigma, rho, epsilon)
optimal_G = model.solve_optimal_G(w, tau)

model = LaborSupplyGraphQ5(alpha, kappa, nu, sigma, rho, epsilon)
tau_range = np.linspace(0, 1, 100)  # Range of tax rates from 0 to 1
model.plot_optimal_G(w, tau_range)

print(f"The optimal G for tau = {tau} is: {optimal_G}")


In [ ]:
from Exam import LaborSupplyGraphQ5

alpha = 0.5
kappa = 1.0
nu = 1.0/(1*16**2)
sigma = 1.5
rho = 1.5
epsilon = 1.0

w = 10.0
tau = 0.5151515151515152

model = LaborSupplyGraphQ5(alpha, kappa, nu, sigma, rho, epsilon)
optimal_G = model.solve_optimal_G(w, tau)

model = LaborSupplyGraphQ5(alpha, kappa, nu, sigma, rho, epsilon)
tau_range = np.linspace(0, 1, 100)  # Range of tax rates from 0 to 1
model.plot_optimal_G(w, tau_range)

print(f"The optimal G for tau = {tau} is: {optimal_G}")

**Question 6:** Find the socially optimal tax rate, $\tau^{\star}$, maximizing worker utility, while keeping $G = \tau w L^{\star}((1-\tau)w,G)$.

In [ ]:
# write your code here  
from Exam import LaborSupplyGraphQ6
alpha = 0.5
kappa = 1.0
nu = 1.0 / (2 * 16 ** 2)
sigma = 1.5
rho = 1.5
epsilon = 1.0

w = 10.0
tau_range = np.linspace(0.0, 1.0, 100)  # Grid of tau values

model = LaborSupplyGraphQ6(alpha, kappa, nu, sigma, rho, epsilon)
model.plot_optimal_G(w, tau_range)

optimal_tax_rate, optimal_G = model.solve_optimal_tax_rate(w)
print("Optimal Tax Rate:", optimal_tax_rate)
print("Corresponding Optimal G:", optimal_G)


## 2. <a id='toc2_'></a>[Problem 2: Labor adjustment costs](#toc0_)

You own a hair salon. You employ hairdressers, $\ell_t$, to produce haircuts, $y_t = \ell_t$.

The wage for each haridresser is $w$.

The demand for haircuts implies that the price of haircuts you can charge is $p_t = \kappa_t y_t^{-\eta}$, where $\kappa_t$ is a demand-shock and $\eta \in (0,1)$ measures the elasticity of demand.

Profits are:

$$
\Pi_t = p_t y_t - w \ell_t = \kappa_t \ell_t^{1-\eta} - w \ell_t
$$

Baseline parameters are:
- $\eta = 0.5$
- $w = 1.0$

**Question 1:** Verify numerically that $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ maximises profits, for $\kappa\in\left\{1.0 , 2.0\right\}$.



In [ ]:
# write your code here
from ExamQ2 import HairSalon
# Import necessary libraries
import sympy as sp

# Instantiate HairSalon object
salon = HairSalon(kappa=1.0, eta=0.5, w=10.0)

# Calculate the expression for the optimal ell
optimal_ell_expr = salon.calculate_optimal_ell()

# Create the equation for the optimal ell
equation = sp.Eq(salon.ell, optimal_ell_expr)

# Print the equation
equation


We now consider a *dynamic* version of the model.

* The demand-shock is a so-called AR(1) in logs, 

$$
\log \kappa_{t} = \rho \log \kappa_{t-1} + \epsilon_{t},\,\,\, \epsilon_{t+1} \sim \mathcal{N}(-0.5\sigma_{\epsilon}^2,\sigma_{\epsilon})
$$

* Any hiring or firing implies a fixed adjustment cost, $\iota > 0 $.
* Future profits are discounted with a monthly factor of $R \in (0,1)$.

The initial demand shock is $\kappa_{-1} = 1$ and the planning horizon is 10 years, i.e. 120 months so $t \in \{0,1,2,\dots,119\}$. Initially you don't have any employees, $\ell_{-1}=0$


The *ex post* value of the salon is *conditional* on the shock series is:

$$
h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119}) = \left[\sum_{t=0}^{119}R^{-t}\left[\kappa_{t}\ell_{t}^{1-\eta}-w\ell_{t}-\boldsymbol{1}_{\ell_{t}\neq\ell_{t-1}}\iota\right]\right]
$$

The *ex ante* expected value of the salon can be approximated by

$$
H = \mathbb{E}[h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119})] \approx \frac{1}{K}\sum_{k=0}^{K} h(\epsilon_0^k,\epsilon_1^k,\dots,\epsilon_{119}^k)
$$

where each $k\in\{0,1,\dots,K-1\}$ is a random shock series. Maximizing profitability means maximizing $H$.


Baseline parameters are: 

- $\rho = 0.90$
- $\iota = 0.01$
- $\sigma_{\epsilon} = 0.10$
- $R = \left(1+0.01\right)^{1/12}$

**Question 2:** Calculate $H$ if the policy  $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ from question 1 is followed. Choose $K$ so the approximation is good enough to not affect your results substantially.

In [ ]:
# write your code here 


from ExamQ2 import HairSalonH


# Set the baseline parameters
rho = 0.90
eta = 0.5
wage = 1.0
iota = 0.01
sigma_epsilon = 0.10
R = (1 + 0.01) ** (1 / 12)

# Create the HairSalon instance
salon = HairSalonH(rho, eta, wage, iota, sigma_epsilon, R)

# Calculate the expected value of the salon
K = 10000  # Number of shock series to simulate
expected_h = salon.calculate_expected_h(K)

print(f"The expected value of the salon (H) is approximately: {expected_h}")


Next, we consider policies on the form:

$$

\ell_{t}=\begin{cases}
\ell_t^{\ast}  & \text{if }\left|\ell_{t-1}-\ell_t^{\ast} \right|>\Delta\\
\ell_{t-1} & \text{else }
\end{cases}
\\
\text{where}\,\,\ell_t^{\ast} = \left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}} \\

$$
With $\Delta \geq 0$ and $\Delta = 0$ being the previous policy.



**Question 3:** Calculate $H$ if the policy above was followed with $\Delta = 0.05$. Does it improve profitability?

In [ ]:
# write your code here

from ExamQ2 import HairSalon3
# Set the baseline parameters
rho = 0.90
eta = 0.5
wage = 1.0
iota = 0.01
sigma_epsilon = 0.10
R = (1 + 0.01) ** (1 / 12)
delta = 0.05

# Create the HairSalon instance with the new policy
salon = HairSalon3(rho, eta, wage, iota, sigma_epsilon, R, delta)

# Calculate the expected value of the salon with the new policy
K = 10000  # Number of shock series to simulate
expected_h_new_policy = salon.calculate_expected_h(K)

print(f"The expected value of the salon with the new policy (H) is approximately: {expected_h_new_policy}")


**Question 4:** Find the optimal $\Delta$ maximizing $H$. Illustrate your result.

In [ ]:
# write your code here

from ExamQ2 import HairSalonoptimal
from scipy.optimize import minimize_scalar
# Set the baseline parameters
rho = 0.90
eta = 0.5
wage = 1.0
iota = 0.01
sigma_epsilon = 0.10
R = (1 + 0.01) ** (1 / 12)

# Create the HairSalon instance
salon = HairSalonoptimal(rho, eta, wage, iota, sigma_epsilon, R)

# Set the optimization bounds for delta
delta_lower_bound = 0.0
delta_upper_bound = 0.1

# Perform optimization
result = minimize_scalar(salon.objective_function, bounds=(delta_lower_bound, delta_upper_bound), method='bounded')

# Retrieve the optimal delta
optimal_delta = result.x
optimal_h = -result.fun

print(f"The optimal delta that maximizes H is approximately: {optimal_delta}")
print(f"The maximum expected value of the salon (H) is approximately: {optimal_h}")


**Question 5:** Suggest an alternative policy you believe might improve profitability. Implement and test your policy.



In [3]:
# write your code here

import numpy as np
from ExamQ2 import HairSalonDynamic
from ExamQ2 import DemandData

# Function to calculate H
def calculate_H(salon, demand_data):
    R = (1 + 0.01) ** (1/12)
    K = 1000  # Number of random shock series

    total_h = 0.0
    for k in range(K):
        h = 0.0
        epsilon_series = demand_data.generate_epsilon_series()  # Generate random shock series
        for t in range(len(epsilon_series)):
            demand_level = epsilon_series[t]
            dynamic_price = salon.calculate_dynamic_price(demand_level)

            # Update h using the dynamic price and other relevant variables
            h += R ** (-t) * dynamic_price

        total_h += h

    H = total_h / K
    return H


# Example usage
base_price = 30.0
salon = HairSalonDynamic(base_price)

num_periods = 120
rho = 0.90
sigma_epsilon = 0.10
demand_data = DemandData(num_periods, rho, sigma_epsilon)

H = calculate_H(salon, demand_data)

# Print the calculated value of H
print("The expected value of H when using dynamic pricing:", H)




The expected value of H when using dynamic pricing: 3277.5838392272444


## 3. <a id='toc3_'></a>[Problem 3: Global optimizer with refined multi-start](#toc0_)

We consider the Griewank function:

$$ f(\boldsymbol{x}) = \sum^n_{i=1} \frac{x^2_i}{4000}-\prod^n_{i=1}\cos\left(\frac{x_i}{\sqrt{i}}\right)+1$$

The **global minimum** of this function is $f(0,0) = 0$ (remember: $\cos(0)=1$).<br>
But the function also have a lot of **local minima**.

In [ ]:
def griewank(x):
    return griewank_(x[0],x[1])
    
def griewank_(x1,x2):
    A = x1**2/4000 + x2**2/4000
    B = np.cos(x1/np.sqrt(1))*np.cos(x2/np.sqrt(2))
    return A-B+1

A **refined global optimizer with multi-start** is:

1. Choose *bounds* for $\mathbf{x}$ and *tolerance* $\tau > 0$.
2. Choose number of *warm-up iterations*, $\underline{K} > 0$ and *maximum number of iterations*, $K > \underline{K}$.
3. In each iteration for $k \in \{0,1,\dots,K-1\}$:

    A. Draw random $\mathbf{x}^k$ uniformly within chosen bounds.

    B. If $k < \underline{K}$ go to step E.

    C. Calculate $\chi^k = 0.50\cdot\frac{2}{1+\exp((k-\underline{K})/100)}$  

    D. Set $\mathbf{x}^{k0} = \chi^k \mathbf{x}^k + (1-\chi^k)\mathbf{x}^{\ast} $

    E. Run optimizer with $\mathbf{x}^{k0}$ as initial guess and $\mathbf{x}^{k\ast}$ as result.

    F. Set $\mathbf{x}^{\ast} = \mathbf{x}^{k\ast}$ if $k = 0$ or $f(\mathbf{x}^{k\ast}) < f(\mathbf{x}^{\ast})$

    G. If $f(\mathbf{x}^{\ast}) < \tau$ go to step 4.

4. Return the result $\mathbf{x}^{\ast}$.

As settings we choose:

* $x_1,x_2 \in  [-600,600]$
* $\tau = 10^{-8}$
* $\underline{K}=10$
* $K=1000$

The optimizer in Step 3.E is `BFGS` with a tolerance of $\tau$.

**Question 1:** Implement the refined global optimizer with multi-start. Illustrate how the effective initial guesses $\mathbf{x}^{k0}$ vary with the iteration counter $k$.

In [ ]:
from ExamQ3 import GriewankOptimizer
# Usage for Question 1
optimizer = GriewankOptimizer(bounds=[-600, 600], tolerance=1e-8, warmup_iters=10, max_iters=1000)
optimizer.optimize()
optimizer.plot_initial_guesses()

**Question 2:** Is it a better idea to set $\underline{K} = 100$? Is the convergence faster?

In [ ]:
from ExamQ3 import GriewankOptimizer
# Usage for Question 1
optimizer = GriewankOptimizer(bounds=[-600, 600], tolerance=1e-8, warmup_iters=100, max_iters=1000)
optimizer.optimize()
optimizer.plot_initial_guesses()